In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sbpy.calib import Sun
from astropy.io import fits

from lvmdrp.core.constants import CON_LAMPS, SPEC_CHANNELS
from lvmdrp.utils import metadata as md
from lvmdrp import config, log, path, __version__ as drpver
from lvmdrp.functions import imageMethod, rssMethod
from lvmdrp.core import tracemask, rss

plt.style.use("seaborn-v0_8-talk")

In [ ]:
flats = md.get_metadata(kind="master", imagetyp="flat")
flats = flats.loc[flats.name.str.contains("ldls") | flats.name.str.contains("quartz")]
flats.sort_values(by="camera", inplace=True)
flats

In [ ]:
# extract fiber flats using traces
lamps = [lamp.lower() for lamp in CON_LAMPS]
cameras = sorted(set(flats.camera))
mfiberflats = []
for _, flat in flats.iterrows():
    # get lamp used
    lamp = flat[lamps].astype(int).idxmax()
    
    # define paths
    mflat = path.full("lvm_master", drpver=drpver, kind=f"mflat_{lamp}", **flat.to_dict())
    xflat = path.full("lvm_master", drpver=drpver, kind=f"xmflat_{lamp}", **flat.to_dict())
    wflat = path.full("lvm_master", drpver=drpver, kind=f"wmflat_{lamp}", **flat.to_dict())
    mfiberflat = path.full("lvm_master", drpver=drpver, kind=f"mfiberflat", **flat.to_dict())
    # define traces path
    mtrace = os.path.join("data_calib", f"lvm-mtrace-{flat.camera}.fits")
    mwave = os.path.join("data_calib", f"lvm-mwave_{'hgne' if flat.camera[0] == 'b' else 'neon'}-{flat.camera}.fits")
    mlsf = os.path.join("data_calib", f"lvm-mlsf_{'hgne' if flat.camera[0] == 'b' else 'neon'}-{flat.camera}.fits")
    iwave, fwave = SPEC_CHANNELS[flat.camera[0]]

    # extract fiberflat
    imageMethod.extract_spectra(in_image=mflat, out_rss=xflat, in_trace=mtrace, method="aperture", aperture=3)
    rssMethod.create_pixel_table(in_rss=xflat, out_rss=wflat, arc_wave=mwave, arc_fwhm=mlsf)
    rssMethod.resample_wavelength(in_rss=wflat, out_rss=wflat, start_wave=iwave, end_wave=fwave, disp_pix=0.5, method="linear", err_sim=10, parallel=0, extrapolate=False)
    
    # compute fiberflats
    rssMethod.create_fiberflat(in_rss=wflat, out_rss=mfiberflat, median_box=0, gaussian_kernel=2.5, display_plots=True)
    mfiberflats.append(mfiberflat)

In [ ]:
new_masters = md.extract_metadata(frames_paths=mfiberflats)
md.add_masters(new_masters)